In [127]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time
import math
from spotipy.exceptions import SpotifyException
from collections import Counter
from IPython.core.interactiveshell import InteractiveShell

# show all columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)


InteractiveShell.ast_node_interactivity = "all"

Audrey's keys: 
* id: 15eeb8c96d3c4e38991a827042884398
* secret: a202e49dc9274a469c7d222428966c1d

Moritz' Keys:
* id: 83578afe5b1d4137a5e813cac12a370a
* secret: 3c4427427adb4a269d0c207b51f3c68d

moritz2:
* id: d8c6553d1c7b4b9781e1ad706e817ad6
* secret: 2b05ae34bbff4e39b7939c09c496075c


In [41]:
# Set up of Spotify API credentials
client_id = "d8c6553d1c7b4b9781e1ad706e817ad6" #Moritz
client_secret = "2b05ae34bbff4e39b7939c09c496075c" #Moritz

# Setting input and output file names
input_file = "billboard_hot_100_2020_2023.csv"
output_file = "billboard_hot_100_audio_features.csv"

# Setting up the Spotify API
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Reading the songs from the input CSV
songs_df = pd.read_csv("Spotify_TT.csv")

In [42]:
# Removing duplicates based on artist name
songs_df = songs_df.drop_duplicates(subset=['artist'], keep='first')

In [43]:
# Erstellen einer Spalte für die Genre
songs_df['Genre API'] = ""

In [44]:
songs_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13624 entries, 0 to 47544
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   track_id          13624 non-null  object 
 1   time_signature    11904 non-null  float64
 2   chorus_hit        11904 non-null  float64
 3   sections          11904 non-null  float64
 4   target            11904 non-null  float64
 5   popularity        1918 non-null   float64
 6   genre             1918 non-null   object 
 7   track             13624 non-null  object 
 8   artist            13624 non-null  object 
 9   duration_ms       13624 non-null  float64
 10  danceability      13624 non-null  float64
 11  energy            13624 non-null  float64
 12  key               13624 non-null  object 
 13  loudness          13624 non-null  float64
 14  mode              13624 non-null  object 
 15  speechiness       13624 non-null  float64
 16  acousticness      13624 non-null  float6

In [45]:
def get_genre_from_uri(uri):
    print("Getting track information...")
    try:
        track = sp.track(uri)
    except SpotifyException as e:
        if e.http_status == 429:
            retry_after = int(e.headers.get('Retry-After', 1))
            print(f"Rate limit exceeded. Waiting for {retry_after} seconds.")
            time.sleep(retry_after)
            return get_genre_from_uri(uri)
        else:
            print(f"Error while getting track information: {e}")
            return ""
    
    artist_id = track["artists"][0]["id"]
    
    print("Getting artist information...")
    artist = sp.artist(artist_id)
    print("Artist information received.")
    
    genres = artist["genres"]
    
    if genres:
        genre_str = ", ".join(genres)
        print(f"Added genres for track URI {uri}: {genre_str}")
        return genre_str
    else:
        print(f"No genres found for track URI {uri}")
        return ""

# Add genres as a new column to your dataframe
def add_genres_with_rate_limit(df, batch_size=1500, start_batch=7):
    num_batches = math.ceil(len(df) / batch_size)

    def process_batch(batch_df):
        for index, row in batch_df.iterrows():
            uri = row['track_id']
            genre_str = get_genre_from_uri(uri)
            print(f"Adding genre '{genre_str}' to DataFrame at index {index}.")
            batch_df.at[index, 'Genre API'] = genre_str
            time.sleep(1 / 3)  # 333 ms
        return batch_df


    for i in range(start_batch, num_batches):
        print(f"Processing batch {i + 1} out of {num_batches}")
        start_index = i * batch_size
        end_index = start_index + batch_size
        df.iloc[start_index:end_index] = process_batch(df.iloc[start_index:end_index])
        
        # Save the processed batch to a CSV file
        df.iloc[start_index:end_index].to_csv(f"V2_batch_{i + 1}.csv", index=False)

add_genres_with_rate_limit(songs_df)


Processing batch 8 out of 10
Getting track information...
Getting artist information...
Artist information received.
Added genres for track URI 2woCC5hiuPTDAonxhEojP0: dance pop, pop
Adding genre 'dance pop, pop' to DataFrame at index 37698.
Getting track information...
Getting artist information...
Artist information received.
Added genres for track URI 034f2qkphFRyX7MmBzC6fr: dansband
Adding genre 'dansband' to DataFrame at index 37699.
Getting track information...
Getting artist information...
Artist information received.
Added genres for track URI 0IkKz2J93C94Ei4BvDop7P: dance pop, pop, pop rap
Adding genre 'dance pop, pop, pop rap' to DataFrame at index 37701.
Getting track information...
Getting artist information...
Artist information received.
Added genres for track URI 6SxAcbydq3I9KXRg6KHJj8: indie dream pop
Adding genre 'indie dream pop' to DataFrame at index 37707.
Getting track information...
Getting artist information...
Artist information received.
Added genres for track 

KeyboardInterrupt: 

In the next step, we need to re-combine the different batches.

In [98]:
import glob

def combine_batches_to_single_file(num_batches, output_filename):
    # Use a list to store each batch DataFrame
    df_list = []

    # Loop through the batches
    for i in range(num_batches):
        # Load the batch into a DataFrame
        df_batch = pd.read_csv(f"V2_batch_{i + 1}.csv")
        
        # Append the batch DataFrame to the list
        df_list.append(df_batch)

    # Concatenate all batch DataFrames
    df_combined = pd.concat(df_list, ignore_index=True)

    # Save the combined DataFrame to a new CSV file
    df_combined.to_csv(output_filename, index=False)

# Get the number of batch files
num_batches = len(glob.glob('V2_batch_*.csv'))

# Combine the batch files into a single file
combine_batches_to_single_file(num_batches, "combined.csv")

combined = pd.read_csv("combined.csv")


In [99]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13624 entries, 0 to 13623
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   track_id          13624 non-null  object 
 1   time_signature    11904 non-null  float64
 2   chorus_hit        11904 non-null  float64
 3   sections          11904 non-null  float64
 4   target            11904 non-null  float64
 5   popularity        1918 non-null   float64
 6   genre             1918 non-null   object 
 7   track             13624 non-null  object 
 8   artist            13624 non-null  object 
 9   duration_ms       13624 non-null  float64
 10  danceability      13624 non-null  float64
 11  energy            13624 non-null  float64
 12  key               13624 non-null  object 
 13  loudness          13624 non-null  float64
 14  mode              13624 non-null  object 
 15  speechiness       13624 non-null  float64
 16  acousticness      13624 non-null  float6

Find out which genres appear the most in the dataset, to select them and build a dictionary to group them.

In [100]:
combined.head()

,track_id,time_signature,chorus_hit,sections,target,popularity,genre,track,artist,duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,era,Genre API
0,1dtKN6wwlolkM8XZy2y9C1,3.0,32.94975,9.0,1.0,NaN,NaN,Jealous Kind Of Fella,Garland Green,173533.0,0.417,0.620,D# / Eb,-7.727,major,0.0403,0.490,0.000000,0.0779,0.845,185.655,60s,"chicago soul, classic soul, southern soul"
1,5hjsmSnUefdUqzsDogisiX,4.0,48.82510,10.0,0.0,NaN,NaN,Initials B.B.,Serge Gainsbourg,213613.0,0.498,0.505,D# / Eb,-12.475,major,0.0337,0.018,0.107000,0.1760,0.797,101.801,60s,"chanson, french rock"
2,6uk8tI6pwxxdVTNlNOJeJh,4.0,37.22663,12.0,0.0,NaN,NaN,Melody Twist,Lord Melody,223960.0,0.657,0.649,F,-13.392,major,0.0380,0.846,0.000004,0.1190,0.908,115.940,60s,calypso
3,7aNjMJ05FvUXACPWZ7yJmv,4.0,24.75484,8.0,0.0,NaN,NaN,Mi Bomba Sonó,Celia Cruz,157907.0,0.590,0.545,G,-12.058,minor,0.1040,0.706,0.024600,0.0610,0.967,105.592,60s,"cha-cha-cha, cuban rumba, mambo, musica tradic..."
4,1rQ0clvgkzWr001POOPJWx,4.0,21.79874,14.0,0.0,NaN,NaN,Uravu Solla,P. Susheela,245600.0,0.515,0.765,B,-3.515,minor,0.1240,0.857,0.000872,0.2130,0.906,114.617,60s,"filmi, vintage tollywood"


In [101]:
# Split the Genre string into a list of genres
combined['Genre_List'] = combined['Genre API'].apply(lambda x: x.split(', ') if isinstance(x, str) else [])

# Expand the list of genres into a series
genre_series = combined['Genre_List'].explode()

# Count the number of occurrences of each genre
genre_counts = genre_series.value_counts()

# Convert the genre counts into a DataFrame
genre_counts_df = genre_counts.reset_index()
genre_counts_df.columns = ['Genre', 'Count']




In [102]:
cutoff = genre_counts_df['Count'].quantile(0.50)
filtered_genres = genre_counts_df[genre_counts_df['Count'] > cutoff]

# Export CSV file
filtered_genres.to_csv("filtered_genres.csv", index=False)

In [103]:
genre_list = {
    "Rap and Hip Hop": ["gangster rap", "southern hip hop", "atl hip hop", "east coast hip hop", "hardcore hip hop", "dirty south rap", "conscious hip hop", "old school atlanta hip hop", "queens hip hop", "g funk", "old school hip hop", "west coast rap", "chicago rap", "crunk", "rap metal", "rap rock", "tagalog rap", "atl trap", "houston rap", "ohio hip hop", "st louis rap", "virginia hip hop", "tennessee hip hop", "dfw rap", "north carolina hip hop", "miami hip hop", "cali rap", "bronx hip hop", "oakland hip hop", "memphis hip hop", "harlem hip hop", "new orleans rap", "philly rap", "detroit hip hop", "toronto rap", "atlanta bass", "chicago punk", "political hip hop", "lgbtq+ hip hop", "canadian hip hop", "uk pop", "pov: indie", "metropopolis", "deep talent show", "bubble trance", "hip house", "trap queen", "viral rap", "meme rap", "indie pop rap", "pop nacional antigas", "viral trap", "atl trap", "pinoy trap"],
    "Pop": ["pop", "dance pop", "pop rap", "pop rock", "pop dance", "post-teen pop", "bubblegum pop", "canadian pop", "pop punk", "pop edm", "pop emo", "uk pop", "pop r&b", "canadian pop", "australian pop", "pop emo", "acoustic pop", "barbadian pop", "pinoy r&b", "pop nacional antigas", "brazilian rock", "classic thai pop", "deep soft rock", "bow pop", "atlanta bass", "viral pop", "indonesian viral pop", "talent show", "torch song", "neo mellow", "neon pop punk", "melodic rap", "alt z", "indietronica", "mellow gold", "mpb", "yacht rock"],
    "Rock": ["soft rock", "rock", "classic rock", "album rock", "heartland rock", "new wave", "alternative rock", "post-grunge", "permanent wave", "glam metal", "modern rock", "punk", "new wave pop", "folk rock", "alternative metal", "pop rock", "hard rock", "country rock", "blues rock", "new romantic", "synthpop", "new wave", "alternative rock", "post-grunge", "permanent wave", "pop rock", "melodic rap", "metal", "nu metal", "pop punk", "punk", "rock-and-roll", "alternative metal", "hard rock", "indie rock", "folk rock", "emo", "modern rock", "post-teen pop", "post-hardcore", "emo rap", "pop emo", "pop punk", "indie pop rap", "emo punk", "easycore"],
    "Electronic Music and Dance": ["edm", "electro house", "big room", "brostep", "progressive electro house", "house", "intelligent dance music", "diva house", "techno", "german techno", "deep house", "filter house", "chicago house", "uk dance", "danish metal", "dutch house", "dance rock", "eurodance", "dance pop", "dancehall", "deep disco", "dance-punk", "viral pop", "electropop", "modern bollywood", "indian classical", "uk post-punk", "indie pop", "indie soul", "alternative dance", "dream pop", "electronica", "indie poptimism", "chillwave", "metropopolis", "neo-synthpop", "indietronica", "chillhop", "lgbtq+ hip hop", "viral trap", "vapor trap", "electronic trap", "electro", "electrofox", "electropop", "electropop", "electro house", "electro swing", "electro-industrial", "escape room", "futurepop", "aggrotech", "electronica", "electronic", "future garage", "glitch hop"],
    "Country and Folk": ["country", "modern country rock", "contemporary country", "country road", "country pop", "outlaw country", "traditional country", "texas country", "red dirt", "new americana", "folk", "folk rock", "british folk", "indie folk", "alternative country", "country rock", "country gospel", "country blues", "americana", "british americana", "australian americana", "country dawn", "roots americana", "heartland rock", "southern rock", "stomp and holler", "traditional folk", "canadian contemporary country", "canadian country", "canadian americana"],
    "Blues and Jazz": ["blues", "classic blues", "modern blues", "delta blues", "traditional blues", "chicago blues", "electric blues", "country blues", "jazz", "cool jazz", "contemporary jazz", "vocal jazz", "free jazz", "jazz blues", "smooth jazz", "latin jazz", "gypsy jazz", "jazz funk", "jazz fusion", "jazz pop", "jazz trio", "classic bollywood", "bolero", "bossa nova", "mpb", "mellow gold", "yacht rock"],
    "World Music": ["world", "world worship", "world meditation", "world chill", "australian dance", "australian electropop", "australian hip hop", "australian pop", "australian reggae fusion", "australian indie", "australian rock", "irish dance", "irish folk", "irish country", "irish neo-traditional", "irish pop", "balkan brass", "british reggae", "classic bollywood", "classic thai pop", "bossa nova", "classic indian pop", "classic pakistani pop", "classical performance", "indian folk", "indian indie", "uk dance", "uk pop", "british folk", "cuban rumba", "latin alternative", "latin metal", "latin rock", "brazilian edm", "brazilian rock", "latin pop", "latin viral pop", "mexican rock-and-roll", "mexican pop", "k-pop"],
    "Classical and Opera": ["classical", "early classical", "late classical", "modern classical", "romantic", "post-romantic", "contemporary classical", "classical performance", "opera", "operatic pop", "classical crossover", "movie tunes", "hollywood", "show tunes", "broadway", "movie tunes", "disney"],
    "Reggae and Ska": ["reggae", "roots reggae", "reggae fusion", "dancehall", "ska", "ska punk", "ska revival", "rocksteady", "jamaican ska", "british reggae", "two tone", "ska mexicano"]
}

In [104]:
# Assuming df is your dataframe
combined['Genre API'] = combined['Genre API'].apply(lambda x: x.split(', ') if isinstance(x, str) else [])

# Create a mapping function
def map_genre(genres):
    parent_genres = []
    for genre in genres:
        for parent_genre, sub_genres in genre_list.items():
            if genre in sub_genres:
                parent_genres.append(parent_genre)
    return parent_genres

# Apply the mapping function to the 'Genre API' column
combined['parent_genre'] = combined['Genre API'].apply(map_genre)

# Function to get the most common parent genre
def most_common(genres):
    if genres:
        return Counter(genres).most_common(1)[0][0]
    else:
        return None

# Apply the most_common function to the 'parent_genre' column
combined['main_parent_genre'] = combined['parent_genre'].apply(most_common)


Extension of the previous dict, as there are currently too litle genres assigned to artists

In [137]:
def add_genre_if_needed(row):
    if not row['main_parent_genre']:
        if any('pop' in genre for genre in row['Genre API']):
            return 'Pop'
        elif any('rap' in genre for genre in row['Genre API']) or any('hip' in genre for genre in row['Genre API']):
            return 'Rap and Hip Hop'
        elif any('rock' in genre for genre in row['Genre API']):
            return 'Rock'
        elif any('r&b' in genre for genre in row['Genre API']):
            return 'Pop'
        elif any('soul' in genre for genre in row['Genre API']):
            return 'Blues and Jazz'
    return row['main_parent_genre']

combined['main_parent_genre'] = combined.apply(add_genre_if_needed, axis=1)


In [139]:
# Export as csv
combined.to_csv('combined.csv', index=False)

In [140]:
# Show counts of each parent genre
print(combined['main_parent_genre'].value_counts())

print("Sum of all assigned parent genres: ", combined['main_parent_genre'].value_counts().sum())

Pop                           2380
Rock                          1870
Rap and Hip Hop               1778
Blues and Jazz                 844
Electronic Music and Dance     809
Country and Folk               514
World Music                    265
Classical and Opera            196
Reggae and Ska                  97
Name: main_parent_genre, dtype: int64
Sum of all assigned parent genres:  8753


In [141]:
# Subset combined to only include rows without a parent genre 
combined_no_parent = combined[combined['main_parent_genre'].isnull()]

# Show counts of each genre
test = combined_no_parent['Genre API'].value_counts()
test.head(20)

[]                                   1737
[freestyle]                            34
[opm]                                  30
[novelty]                              23
[deep adult standards]                 23
[slowed and reverb]                    20
[doo-wop, rhythm and blues]            20
[doo-wop]                              19
[library music]                        18
[gospel]                               18
[vintage italian soundtrack]           18
[covertronica]                         15
[british invasion]                     15
[disco]                                14
[idol]                                 13
[poetry]                               13
[adult standards, easy listening]      13
[easy listening]                       12
[asmr]                                 12
[nashville sound]                      12
Name: Genre API, dtype: int64

In [143]:
# Adding Other to the parent genre column for all rows without a parent genre in combined dataframe
combined.loc[combined['main_parent_genre'].isnull(), 'main_parent_genre'] = 'Other'

combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13624 entries, 0 to 13623
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   track_id           13624 non-null  object 
 1   time_signature     11904 non-null  float64
 2   chorus_hit         11904 non-null  float64
 3   sections           11904 non-null  float64
 4   target             11904 non-null  float64
 5   popularity         1918 non-null   float64
 6   genre              1918 non-null   object 
 7   track              13624 non-null  object 
 8   artist             13624 non-null  object 
 9   duration_ms        13624 non-null  float64
 10  danceability       13624 non-null  float64
 11  energy             13624 non-null  float64
 12  key                13624 non-null  object 
 13  loudness           13624 non-null  float64
 14  mode               13624 non-null  object 
 15  speechiness        13624 non-null  float64
 16  acousticness       136